In [1]:
import streamlit as st
import numpy as np
import pandas as pd
from PIL import Image
import gspread
import json
import pandas as pd
#ServiceAccountCredentials：Googleの各サービスへアクセスできるservice変数を生成します。
from oauth2client.service_account import ServiceAccountCredentials

#2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
json_path = 'kitaichi-fd2dd716665b.json'
#認証情報設定
#ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
#OAuth2の資格情報を使用してGoogle APIにログインします。
gc = gspread.authorize(credentials)
# １．ファイル名を指定してワークブックを選択
workbook = gc.open('収支表')
sh_shop = workbook.worksheet('店一覧')
sh_slot_kisyu = workbook.worksheet('スロット機種一覧')
df_shop = pd.DataFrame(sh_shop.get_all_values()[1:], columns=sh_shop.get_all_values()[0])
df_slot_kisyu = pd.DataFrame(sh_slot_kisyu.get_all_values()[1:], columns=sh_slot_kisyu.get_all_values()[0])
sh = workbook.worksheet('収支')
df = pd.DataFrame(sh.get_all_values()[1:], columns=sh.get_all_values()[0])

In [2]:
df.tail()

,年月日,開始時間,終了時間,店舗名,台番号,機種名,開始G数,当選G数,ヤメG数,投資金額,投資メダル枚数,回収金額,回収メダル枚数,期待値,収支金額,収支メダル枚数,備考
13,2022-12-13,19:48:05,20:46:37,ダイナム小郡店,361,パチスロ戦姫絶唱シンフォギア 勇気の歌,208,621,0,13120,644,8226,436,500,-4894,.................................................,
14,2022-12-14,17:58:39,18:12:53,ダイナム宇部港町店,164,Sスーパー海物語 IN JAPAN祭,257,239,0,1000,100,960,96,900,-40,-4,
15,2022-12-14,19:19:59,19:57:00,メガガイア小郡店,1128,パチスロ戦姫絶唱シンフォギア 勇気の歌,294,337,0,1000,160,3250,650,400,2250,490,
16,2022-12-14,20:00:27,20:27:23,メガガイア小郡店,1610,沖ドキ!DUO,427,623,0,7360,368,4307.692308,224,100,-3053,-144,0スルー、1スルー狙い
17,2022-12-14,20:31:20,22:02:00,メガガイア小郡店,1610,沖ドキ!DUO,0,67,0,2038,102,46980,2443,1000,44942,2341,B以上確定示唆あり


In [93]:
days_list=[]
day_list = list(set(df['年月日']))
for day in day_list:
    day_df = df[df['年月日']==day]
    sum([int(i) for i in list(day_df['期待値'])])
    sum([int(i) for i in list(day_df['収支金額'])])
    days_list.append([day,sum([int(i) for i in list(day_df['期待値'])]),sum([int(i) for i in list(day_df['収支金額'])])])
day_df = pd.DataFrame(days_list,columns=['年月日','期待値','収支金額']).sort_values('年月日').reset_index(drop=True)
suii_list = []
kitaiti_sum = 0
syushi_sum = 0
for i in range(len(list(day_df['期待値']))):
    if i == 0:
        kitaiti_sum = list(day_df['期待値'])[i]
        syushi_sum = list(day_df['収支金額'])[i]
    else:
        kitaiti_sum = list(day_df['期待値'])[i] + kitaiti_sum
        syushi_sum = list(day_df['収支金額'])[i] + syushi_sum
    
    suii_list.append([list(day_df['年月日'])[i],kitaiti_sum,syushi_sum])
suii_df = pd.DataFrame(suii_list,columns=['年月日','期待値','収支金額'])

['2022-12-05', '2022-12-09', '2022-12-02', '2022-12-06']


In [94]:
suii_df

,年月日,期待値,収支金額
0,2022-12-02,2700,4320
1,2022-12-05,4295,2810
2,2022-12-06,4695,7830
3,2022-12-09,6195,3870
